# Lab 3 Multi-Task Model
## 03/20/2020
## Zekai Wang, Xinqun Ye

In [3]:
import numpy as np

f = open('dataCHEMBL/data.csv', 'r', encoding="ISO-8859-1")
lines = f.readlines()
f.close()


flis = [] # will have [id, ic50_val, unit, target, smiles] 

for line in lines[1:]: # first line contains the headers (therefore skiped)
    lis = line.strip().split(',')

    if len(lis) < 36:
        continue
    
    
    #print (lis[0], lis[3], lis[8], lis[36])
    # [id, ic50_val, unit, target, smiles]

    flis.append([lis[0], lis[3], lis[4], lis[8], lis[36]])
    #break

# I have no idea why python adds \x00 to each string
# Therefore I removed it

for i in range (len(flis)):
    flis[i] = [j.replace('\x00','') for j in flis[i]]
    flis[i] = [j.replace('"','') for j in flis[i]]

# Some of the data have smiles or IC50 missing
# So I removed them as well
list1 = []
for i in flis:
    if len(i[1]) != 0 and len(i[-1]) != 0:
        #print (i)
        list1.append(i)

X, t, y = [], [], []
for i in list1:
    if i[2] == 'nM':
        if len(i[-1]) == 0 or len(i[-2]) ==0 or len(i[1]) ==0:
            print ('Thers a problem !!')
        X.append(i[-1])
        t.append(i[-2])
        y.append(i[1])

# smiles is the string representation of each ligand
# target is the protein where ligand binds (treat it as different schools)
# ic50 is the score

We can notice there are 67184 unique target in the our dataset. After we counted what number is for each targets. We find there are 100 targets who has ligand greater than 334. Therefore, we filter those 100 targets as our top 100 targets.

In [4]:
print('There is %d unique target in our dataset' %len(set(t)))

There is 67184 unique target in our dataset


In [5]:
#count by dictionary
dict1 = {}
for key in t:
    dict1[key] = dict1.get(key,0) + 1
    
print('The 100th target(sorted by descending) has %d ligand'
      %sorted(dict1.items(), key=lambda item:item[1],reverse=True)[99][1])

The 100th target(sorted by descending) has 334 ligand


In [6]:
#created a dataframe for smiles, target and ic50
import pandas as pd
d = {'smiles':X, 'target':t, 'ic50':y}
df1 = pd.DataFrame(d)
df1['ic50'] = df1['ic50'].astype('float')

In [7]:
#get filtered dataframe
df2 = pd.DataFrame()
for i in range(len(df1['target'])):
    if dict1[df1['target'][i]] > 333:
        df2 = df2.append(df1[i:i+1])

In [8]:
df2['index'] = range(df2.shape[0])
df2 = df2.set_index('index')

In [9]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59839 entries, 0 to 59838
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   smiles  59839 non-null  object 
 1   target  59839 non-null  object 
 2   ic50    59839 non-null  float64
dtypes: float64(1), object(2)
memory usage: 1.8+ MB


In [10]:
df2.head()

,smiles,target,ic50
index,,,
0,COc1ccccc1CNC(=O)CCN2C(=O)Nc3ccccc3C2=O,CHEMBL3436040,12000.0
1,Cl.COc1ccc2nccc(C(O)C3CC4(CCN3CC4)C=C)c2c1,CHEMBL3436040,12000.0
2,Cn1c(nc2ccccc12)C(=O)c3cccc4ccccc34,CHEMBL3436040,12000.0
3,Cl.CN1C(=O)N(C)c2cc(CNCCNc3ccnc4cc(Cl)ccc34)ccc12,CHEMBL3436040,12000.0
4,COc1ccc(CNC(=O)C2=CC3=C(N=C4C=CC=CN4C3=O)N(CCc...,CHEMBL3436040,12000.0


## Binarize 
After we removed the instance whose ic50 between 300 and 10000, we have 17965 instances. Then, we binarize the ic50 into "active" and "inactive"

In [11]:
df3 = pd.DataFrame()
for i in range(len(df2['target'])):
    if df2['ic50'][i] <= 300 or df2['ic50'][i] >= 10000:
        df3 = df3.append(df2[i:i+1])

In [12]:
df3['index'] = range(df3.shape[0])
df3 = df3.set_index('index')

In [13]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45440 entries, 0 to 45439
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   smiles  45440 non-null  object 
 1   target  45440 non-null  object 
 2   ic50    45440 non-null  float64
dtypes: float64(1), object(2)
memory usage: 1.4+ MB


In [14]:
df3.tail()

,smiles,target,ic50
index,,,
45435,1.76,CHEMBL3887886,66.0
45436,9606,CHEMBL3887679,9.9
45437,S=P(N1CC1)(N2CC2)N3CC3,CHEMBL4017550,1000000.0
45438,CN(C)S(=O)(=O)c1ccc2Sc3ccccc3\C(=C\CCN4CCN(C)C...,CHEMBL4017550,30400.0
45439,CCS[C@@]1(CC[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C...,CHEMBL4017550,10000.0


In [29]:
for i in range(len(df3['target'])):
    if df3.loc[i,'ic50'] >= 10000:
        df3.loc[i,'Binary'] = 1
    else:
        df3.loc[i,'Binary'] = 0       

In [30]:
df3.tail(1000)

,smiles,target,ic50,Binary
index,,,,
44440,0.29,CHEMBL3888383,25.5,0
44441,3.4,CHEMBL3887759,100.0,0
44442,HEK293,CHEMBL3889097,23.6,0
44443,HEK293,CHEMBL3889097,13.3,0
44444,Sunnyvale Calif.) equipped with voltage-senso...,CHEMBL3889155,20000.0,1
...,...,...,...,...
45435,1.76,CHEMBL3887886,66.0,0
45436,9606,CHEMBL3887679,9.9,0
45437,S=P(N1CC1)(N2CC2)N3CC3,CHEMBL4017550,1000000.0,1


##  Featurize each ligand
Since we have each single "smiles", we chose "Chem.MolFromSmiles" function to construct individual molecules. The result will return a rdkit.Chem.rdchem.Mol object, if successful, or None, if fail.  
Then, we use Morgan Fingerprints (Circular Fingerprints) to calculate our molecules as bit vectors. For the "GetMorganFingerprintAsBitVect" function, we use radius = 2 and nBits = 1024.

Reference: https://www.researchgate.net/post/How_to_calculate_circular_fingerprints_using_RDKit  


In [31]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.AtomPairs import Pairs

In [32]:
m = Chem.MolFromSmiles(df3['smiles'][100])
AllChem.GetMorganFingerprintAsBitVect(m,2,nBits=1024)
#Pairs.GetAtomPairFingerprintAsBitVect(m)

In [33]:
#https://www.researchgate.net/post/How_to_calculate_circular_fingerprints_using_RDKit

BinVecFeature, Error = [], []

def GenerateVec(smiles):
    m = Chem.MolFromSmiles(smiles)
    
    if m is None:
        raise ValueError('This "smiles" cannot be converted into individual molecules', smiles)
#    return Pairs.GetAtomPairFingerprintAsBitVect(m)

    return AllChem.GetMorganFingerprintAsBitVect(m, radius=2, nBits=1024)

for i in range (len(df3['smiles'])):
    try:
        st = GenerateVec(df3['smiles'][i]).ToBitString()
    except ValueError:
        Error.append(i)
        
    l = [int(j) for j in st]
    BinVecFeature.append(l)
    
BinVecFeature = np.array(BinVecFeature)
Error = set(Error) #13985 unique Error
print (BinVecFeature.shape)

#        m = Chem.MolFromSmiles(df3['target'][i])
#        if m is None:
#            raise ValueError('This "smiles" cannot be converted into individual molecules', df3['target'][i])
#        st = AllChem.GetMorganFingerprintAsBitVect(m, radius=2, nBits=512).ToBitString()
#        #st = Pairs.GetAtomPairFingerprintAsBitVect(m)

(45440, 1024)


In [34]:
X, task, score = [], [], []
for i in range (len(df3['target'])):
    if i in Error:
        continue
    X.append(BinVecFeature[i])
    task.append(df3['target'][i])
    score.append(df3['Binary'][i])

        

dict3 = {}
for i in range (len(task)):
    if task[i] in dict3:
        dict3[task[i]].append(i)
    else:
        dict3[task[i]] = [i]
        

In [35]:
X = np.array(X)
y = np.array(score).reshape((-1, 1))
print("We have an input array of shape:", X.shape)
print("We have an output array of shape:", y.shape)

We have an input array of shape: (31455, 1024)
We have an output array of shape: (31455, 1)


In [36]:
#count by dictionary
#here we only have 65 unique task
dict2 = {}
for key in task:
    dict2[key] = dict2.get(key,0) + 1
print(dict2)

{'CHEMBL3436040': 423, 'CHEMBL1614547': 423, 'CHEMBL1614549': 305, 'CHEMBL1614359': 276, 'CHEMBL1119333': 314, 'CHEMBL1614423': 381, 'CHEMBL1614063': 740, 'CHEMBL1614432': 621, 'CHEMBL3431931': 369, 'CHEMBL2114719': 339, 'CHEMBL2114714': 1346, 'CHEMBL3436039': 428, 'CHEMBL3436041': 411, 'CHEMBL1738498': 647, 'CHEMBL1963965': 333, 'CHEMBL1964000': 367, 'CHEMBL2114818': 369, 'CHEMBL2114881': 2026, 'CHEMBL1613799': 571, 'CHEMBL1614199': 367, 'CHEMBL1614529': 164, 'CHEMBL1738424': 557, 'CHEMBL1006579': 271, 'CHEMBL1613885': 619, 'CHEMBL3706327': 475, 'CHEMBL3707950': 265, 'CHEMBL3705123': 577, 'CHEMBL3887033': 289, 'CHEMBL2354289': 152, 'CHEMBL2218781': 598, 'CHEMBL1963883': 1184, 'CHEMBL1964096': 373, 'CHEMBL1963941': 1185, 'CHEMBL1738407': 512, 'CHEMBL2114858': 170, 'CHEMBL3734252': 239, 'CHEMBL3562022': 708, 'CHEMBL3705306': 35, 'CHEMBL3431934': 391, 'CHEMBL1963971': 93, 'CHEMBL3734174': 299, 'CHEMBL3734165': 214, 'CHEMBL3880334': 390, 'CHEMBL3880346': 219, 'CHEMBL3880345': 550, 'CHEMBL

## Multi-Task Model

In [37]:
import scipy.io
from sklearn.datasets.base import Bunch
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

In [38]:
tasks = {}

for i in dict3:
    descr = "CHEM data: Target {}".format(i)
    tid = "Target {}".format(i)
#    tasks.append(Bunch(data=X[dict3[i]],
#                       target=y[dict3[i]],
#                       DESCR=descr,
#                       ID=tid))
    tasks[tid] = Bunch(data=X[dict3[i]],
                       target=y[dict3[i]],
                       DESCR=descr)

In [39]:
from keras.layers import Input, Dense
from keras.models import Model

inputs = Input(shape=(X[0].shape[0],))
shared_mlp = Dense(64, activation='relu')(inputs)
shared_mlp = Dense(64, activation='relu')(shared_mlp)

# make a dictionary of models for each task
models_mtl = dict()

for i in tasks.keys():
    personal_layer = Dense(10, activation='relu')(shared_mlp)
    output_layer = Dense(1)(personal_layer)
    model = Model(inputs=inputs, outputs=output_layer)
    model.compile(optimizer='sgd', loss='binary_crossentropy')
    models_mtl[i] = model

In [40]:
X_train = dict()
y_train = dict()
X_test = dict()
y_test = dict()

for x in tasks.keys():
    X_train[x], X_test[x], y_train[x], y_test[x] = train_test_split(tasks[x].data, 
                                                        tasks[x].target, 
                                                        test_size=0.2, 
                                                        random_state=42)

In [41]:
%%time
EPOCHS = 20
from random import shuffle

all_keys = list(tasks.keys())
for i in range(EPOCHS):
    print('Epoch',i,end=', ')
    shuffle(all_keys)
    for x in all_keys:
        models_mtl[x].fit(X_train[x],y_train[x], verbose=0)

Epoch 0, Epoch 1, Epoch 2, Epoch 3, Epoch 4, Epoch 5, Epoch 6, Epoch 7, Epoch 8, Epoch 9, Epoch 10, Epoch 11, Epoch 12, Epoch 13, Epoch 14, Epoch 15, Epoch 16, Epoch 17, Epoch 18, Epoch 19, CPU times: user 1min 26s, sys: 11 s, total: 1min 37s
Wall time: 1min 13s


In [42]:
%%time
y_mtl=[]
yhat_mtl=[]

for x in X_test.keys():
    yhat_mlp = models_mtl[x].predict(X_test[x])

    y_mtl.append(y_test[x].ravel())
    yhat_mtl.append(yhat_mlp.ravel())

CPU times: user 1.74 s, sys: 215 ms, total: 1.95 s
Wall time: 1.77 s
